In [25]:
import pandas as pd

from sup_data_to_fasta import load_xlsx, map_raw_data_to_queries_and_expected

In [26]:
esp_df = pd.read_csv('../output/ESP-OD2w11_results.csv').drop(columns=['Unnamed: 0'])

In [27]:
total_alignments = len(esp_df[esp_df['qseqid'].notnull()])
not_aligned = len(esp_df[esp_df['qseqid'].isnull()])
print(f"Number of alignments in ESP-OD2w11: {total_alignments}")
print(f"Not aligned in ESP-OD2w11: {not_aligned}")

Number of alignments in ESP-OD2w11: 262601
Not aligned in ESP-OD2w11: 0


In [28]:
len(esp_df[esp_df['sseq_cds'] == esp_df['sseq_genome']])

11912

In [29]:
cds_columns = esp_df.filter(like='_cds')
cds_columns


,qgi_cds,qacc_cds,qaccver_cds,qlen_cds,sseqid_cds,sallseqid_cds,sgi_cds,sallgi_cds,sacc_cds,saccver_cds,...,staxids_cds,sscinames_cds,scomnames_cds,sblastnames_cds,sskingdoms_cds,stitle_cds,salltitles_cds,sstrand_cds,qcovs_cds,qcovhsp_cds
0,0.0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
1,0.0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212.0,lcl|NC_016810.1_cds_WP_001520210.1_484,lcl|NC_016810.1_cds_WP_001520210.1_484,0.0,0.0,lcl|NC_016810.1_cds_WP_001520210.1_484,lcl|NC_016810.1_cds_WP_001520210.1_484,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,plus,13.0,13.0
2,0.0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212.0,lcl|NC_017718.1_cds_WP_000142425.1_4636,lcl|NC_017718.1_cds_WP_000142425.1_4636,0.0,0.0,lcl|NC_017718.1_cds_WP_000142425.1_4636,lcl|NC_017718.1_cds_WP_000142425.1_4636,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,plus,7.0,7.0
3,0.0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212.0,lcl|NC_016810.1_cds_WP_000809815.1_3388,lcl|NC_016810.1_cds_WP_000809815.1_3388,0.0,0.0,lcl|NC_016810.1_cds_WP_000809815.1_3388,lcl|NC_016810.1_cds_WP_000809815.1_3388,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,plus,8.0,8.0
4,0.0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,0.0,0.0,lcl|NC_016810.1_cds_WP_001575544.1_1,lcl|NC_016810.1_cds_WP_001575544.1_1,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,lcl|NC_016810.1_cds_WP_001575544.1_1 [gene=thr...,plus,31.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262596,0.0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65.0,lcl|NC_016810.1_cds_WP_001181280.1_854,lcl|NC_016810.1_cds_WP_001181280.1_854,0.0,0.0,lcl|NC_016810.1_cds_WP_001181280.1_854,lcl|NC_016810.1_cds_WP_001181280.1_854,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,plus,20.0,20.0
262597,0.0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65.0,lcl|NC_016810.1_cds_WP_000057014.1_677,lcl|NC_016810.1_cds_WP_000057014.1_677,0.0,0.0,lcl|NC_016810.1_cds_WP_000057014.1_677,lcl|NC_016810.1_cds_WP_000057014.1_677,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,minus,29.0,29.0
262598,0.0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65.0,lcl|NC_016810.1_cds_WP_001526312.1_392,lcl|NC_016810.1_cds_WP_001526312.1_392,0.0,0.0,lcl|NC_016810.1_cds_WP_001526312.1_392,lcl|NC_016810.1_cds_WP_001526312.1_392,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,plus,25.0,25.0
262599,0.0,f974ff36-c05a-11ee-ab2c-0242ac110002,f974ff36-c05a-11ee-ab2c-0242ac110002,51.0,lcl|NC_016810.1_cds_WP_010988985.1_824,lcl|NC_016810.1_cds_WP_010988985.1_824,0.0,0.0,lcl|NC_016810.1_cds_WP_010988985.1_824,lcl|NC_016810.1_cds_WP_010988985.1_824,...,0.0,NaN,NaN,NaN,NaN,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,plus,33.0,33.0


In [30]:
esp_df['gene'] = esp_df['stitle_cds'].str.extract(r'gene=(\w+)')
esp_df['locus_tag'] = esp_df['stitle_cds'].str.extract(r'locus_tag=(\w+)')
esp_df['expected_gene'] = esp_df['expected'].str.extract(r'(\w+)\(')
esp_df['expected_locus_tag'] = esp_df['expected'].str.extract(r'(\w*\()?(.*\))?')


ValueError: Cannot set a DataFrame with multiple columns to the single column expected_locus_tag

In [ ]:
esp_df['qseqid'].nunique()

1252

In [ ]:
def no_expected_match(df):
    
    grouped = df.groupby('query_id')
    count_alignments = grouped[grouped['expected_gene'] == grouped['gene']].count()
    
    return count_alignments

,qseqid,qgi_genome,qacc_genome,qaccver_genome,qlen_genome,sseqid_genome,sallseqid_genome,sgi_genome,sallgi_genome,sacc_genome,...,stitle_cds,salltitles_cds,sstrand_cds,qcovs_cds,qcovhsp_cds,expected,query_id,gene,locus_tag,expected_gene
1,f970f166-c05a-11ee-ab2c-0242ac110002,0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,plus,13.0,13.0,thrL(SL1344_0001),f970f166-c05a-11ee-ab2c-0242ac110002,htpG,SL1344_RS02480,thrL
2,f970f166-c05a-11ee-ab2c-0242ac110002,0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,plus,7.0,7.0,thrL(SL1344_0001),f970f166-c05a-11ee-ab2c-0242ac110002,NaN,SL1344_RS24260,thrL
3,f970f166-c05a-11ee-ab2c-0242ac110002,0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,lcl|NC_016810.1_cds_WP_000809815.1_3388 [gene=...,plus,8.0,8.0,thrL(SL1344_0001),f970f166-c05a-11ee-ab2c-0242ac110002,arcB,SL1344_RS17190,thrL
5,f970f166-c05a-11ee-ab2c-0242ac110002,0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,lcl|NC_016810.1_cds_WP_001520210.1_484 [gene=h...,plus,13.0,13.0,thrL(SL1344_0001),f970f166-c05a-11ee-ab2c-0242ac110002,htpG,SL1344_RS02480,thrL
6,f970f166-c05a-11ee-ab2c-0242ac110002,0,f970f166-c05a-11ee-ab2c-0242ac110002,f970f166-c05a-11ee-ab2c-0242ac110002,212,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,lcl|NC_017718.1_cds_WP_000142425.1_4636 [locus...,plus,7.0,7.0,thrL(SL1344_0001),f970f166-c05a-11ee-ab2c-0242ac110002,NaN,SL1344_RS24260,thrL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262596,f974feaa-c05a-11ee-ab2c-0242ac110002,0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,lcl|NC_016810.1_cds_WP_001181280.1_854 [locus_...,plus,20.0,20.0,ArcZ(ncRNA0002),f974feaa-c05a-11ee-ab2c-0242ac110002,NaN,SL1344_RS04385,ArcZ
262597,f974feaa-c05a-11ee-ab2c-0242ac110002,0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,lcl|NC_016810.1_cds_WP_000057014.1_677 [gene=t...,minus,29.0,29.0,ArcZ(ncRNA0002),f974feaa-c05a-11ee-ab2c-0242ac110002,tcuC,SL1344_RS03480,ArcZ
262598,f974feaa-c05a-11ee-ab2c-0242ac110002,0,f974feaa-c05a-11ee-ab2c-0242ac110002,f974feaa-c05a-11ee-ab2c-0242ac110002,65,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,lcl|NC_016810.1_cds_WP_001526312.1_392 [gene=s...,plus,25.0,25.0,ArcZ(ncRNA0002),f974feaa-c05a-11ee-ab2c-0242ac110002,sbcC,SL1344_RS02020,ArcZ
262599,f974ff36-c05a-11ee-ab2c-0242ac110002,0,f974ff36-c05a-11ee-ab2c-0242ac110002,f974ff36-c05a-11ee-ab2c-0242ac110002,51,NC_016810.1,NC_016810.1,0,0,NC_016810.1,...,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,lcl|NC_016810.1_cds_WP_010988985.1_824 [locus_...,plus,33.0,33.0,STnc740(ncRNA0194),f974ff36-c05a-11ee-ab2c-0242ac110002,NaN,SL1344_RS04245,STnc740
